In [1]:
from importlib import reload
import sys
import util
reload(util)
data = util.FontData
FontDataset = util.FontDataset
import experiment
reload(experiment)
from sklearn import linear_model, neural_network

In [29]:
data.load(knn_path="data/svm_dataset.csv")

Loading embeddings...done
Loading typographic + semantic vectors...done
Loading images...done
Loading glyphs...done
Loading SVGs...done


In [30]:
train_dataset = FontDataset(data, 'train')
val_dataset = FontDataset(data, 'val')

In [31]:
train_typographic = data.get_all_typographic('train')
val_typographic = data.get_all_typographic('val')

In [32]:
train_image = data.get_all_image('train')
train_image = train_image.reshape(train_image.shape[0], -1)
val_image = data.get_all_image('val')
val_image = val_image.reshape(val_image.shape[0], -1)

In [33]:
train_semantic = data.get_all_semantic('train')
val_semantic = data.get_all_semantic('val')

### Linear Regression: Image -> Typographic

In [34]:
image_typographic_reg = linear_model.LinearRegression()

In [35]:
image_typographic_reg.fit(train_image, train_typographic)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [36]:
train_stats, train_typographic_pred = experiment.run_experiment(image_typographic_reg, train_image, train_typographic)

{'RMSE': 2.0718048290291335e-06, 'R2': 0.9999999996011757}


In [37]:
val_stats, val_typographic_pred = experiment.run_experiment(image_typographic_reg, val_image, val_typographic)

{'RMSE': 0.12268583476543427, 'R2': 0.429673389702723}


### Linear Regression: Typographic -> Semantic

In [38]:
typographic_semantic_reg = linear_model.LinearRegression()

In [39]:
typographic_semantic_reg.fit(train_typographic, train_semantic)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [40]:
train_stats, train_semantic_pred = experiment.run_experiment(typographic_semantic_reg, train_typographic, train_semantic)

{'RMSE': 0.1139225885272026, 'R2': 0.4005594469602474}


In [41]:
val_stats, val_semantic_pred = experiment.run_experiment(typographic_semantic_reg, val_typographic, val_semantic)

{'RMSE': 0.1155150830745697, 'R2': 0.43179182329209054}


### Linear Regressions: Image -> Typographic -> Semantic

In [42]:
class SequentialLinearRegression:
    def __init__(self, reg1, reg2):
        self.reg1 = reg1
        self.reg2 = reg2
    def predict(self, X):
        return self.reg2.predict(self.reg1.predict(X))

image_typographic_semantic_reg = SequentialLinearRegression(image_typographic_reg, typographic_semantic_reg)

In [43]:
train_semantic

array([[0.32310462, 0.27001329, 0.46351598, ..., 0.47193706, 0.66785664,
        0.55996615],
       [0.5226276 , 0.53704704, 0.40875095, ..., 0.68417439, 0.52153146,
        0.47034872],
       [0.24908079, 0.12160715, 0.26008224, ..., 0.71686523, 0.45114671,
        0.43598064],
       ...,
       [0.73103569, 0.70339923, 0.53434578, ..., 0.51954736, 0.7770573 ,
        0.45499803],
       [0.75992842, 0.63850497, 0.47036837, ..., 0.49140267, 0.50637171,
        0.36953215],
       [0.71492412, 0.8191593 , 0.46324267, ..., 0.73498388, 0.51533687,
        0.34235952]])

In [44]:
train_stats, train_semantic_pred = experiment.run_experiment(image_typographic_semantic_reg, train_image, train_semantic)

{'RMSE': 0.11392255127429962, 'R2': 0.4005597566616666}


In [45]:
val_stats, val_semantic_pred = experiment.run_experiment(image_typographic_semantic_reg, val_image, val_semantic)

{'RMSE': 0.12904515862464905, 'R2': 0.29791234583298165}


### Linear Regression: Image -> Semantic

In [46]:
image_semantic_reg = linear_model.LinearRegression()

In [47]:
image_semantic_reg.fit(train_image, train_semantic)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [48]:
train_stats, train_semantic_pred = experiment.run_experiment(image_semantic_reg, train_image, train_semantic)

{'RMSE': 1.6354473473256803e-06, 'R2': 0.9999999998826672}


In [49]:
val_stats, val_semantic_pred = experiment.run_experiment(image_semantic_reg, val_image, val_semantic)

{'RMSE': 0.08402460068464279, 'R2': 0.6787419649517175}


### Multilayer Perceptron: Image -> Semantic

In [50]:
image_semantic_nn = neural_network.MLPRegressor(hidden_layer_sizes=(10,))

In [51]:
image_semantic_nn.fit(train_image, train_semantic)

//anaconda/envs/py229/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(10,), learning_rate='constant',
             learning_rate_init=0.001, max_iter=200, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=None, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False)

In [52]:
train_stats, train_semantic_pred = experiment.run_experiment(image_semantic_nn, train_image, train_semantic)

{'RMSE': 0.18476077914237976, 'R2': -0.7886660223638159}


In [53]:
val_stats, val_semantic_pred = experiment.run_experiment(image_semantic_nn, val_image, val_semantic)

{'RMSE': 0.1925349086523056, 'R2': -0.7100480658255814}


#### Example

In [54]:
val_semantic_pred[0]

tensor([0.4038, 0.2929, 0.4468, 0.4670, 0.1925, 0.4957, 0.7383, 0.4120, 0.3113,
        0.3000, 0.3780, 0.2835, 0.3789, 0.6255, 0.7377, 0.4742, 0.6981, 0.4352,
        0.4562, 0.6853, 0.2488, 0.3907, 0.4056, 0.5464, 0.3543, 0.5231, 0.3234,
        0.3285, 0.5067, 0.5747, 0.4867])

In [55]:
data.get_semantic(data.get_name(0, "val"))

array([0.66309171, 0.74742591, 0.39773298, 0.42871607, 0.78757512,
       0.81113136, 0.71726259, 0.68604096, 0.73739551, 0.78661307,
       0.54058519, 0.66010776, 0.64248944, 0.34194715, 0.59816354,
       0.58899834, 0.76675712, 0.52296749, 0.45709049, 0.27298147,
       0.58858916, 0.67330298, 0.42800172, 0.43713958, 0.68671979,
       0.80027663, 0.21062001, 0.46937317, 0.72027824, 0.3953031 ,
       0.29215768])